回顧 sequential

##### Sequential
---------------
- 神經網路層線性堆疊的建模方式
- 只需專注於建立神經網路層
- 當某層需修改，所有層都需重新定義

sequential 的不同面向

初始準備

In [1]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# keras.functions
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD  #notice! use tensorflow.keras.optimizers

# keras.dataset
from keras.datasets import mnist  # 手寫辨識資料  mnist

# keras.utilis function
from keras.utils import np_utils    # one-Hot encoding 0 ~ 9 改成向量的 utils

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
print('共有 %d 訓練資料, 每筆資料尺寸為 %d x %d' %x_train.shape)
print('共有 %d 測試資料, 每筆資料尺寸為 %d x %d' %x_test.shape)

共有 60000 訓練資料, 每筆資料尺寸為 28 x 28
共有 10000 測試資料, 每筆資料尺寸為 28 x 28


In [6]:
x_train = x_train.reshape(60000, 784)  # 矩陣拉平
x_test = x_test.reshape(10000, 784)

In [7]:
x_train_01 = x_train[y_train <= 1]  #將資料中 0, 1 複製一份出來
x_test_01 = x_test[y_test <= 1]

In [8]:
y_train_10 = np_utils.to_categorical(y_train, 10)
y_test_10 = np_utils.to_categorical(y_test, 10)

y_train_01 = y_train[y_train <= 1]
y_train_01 = np_utils.to_categorical(y_train_01, 2)

y_test_01 = y_test[y_test <= 1]
y_test_01 = np_utils.to_categorical(y_test_01, 2)

將 label 轉成 one-Hot encoding 形式

In [9]:
[x_train_01.shape, y_train_01.shape, x_test_01.shape,  y_test_01.shape]

[(12665, 784), (12665, 2), (2115, 784), (2115, 2)]

In [10]:
# Construct a sandbox to put layers inside

model = Sequential()

# Put fully-connected layers(Dense) inside建立一個空的盒子

model.add(Dense(500, input_dim = 784))
model.add(Activation('sigmoid'))
model.add(Dense(500))
model.add(Activation('sigmoid'))
model.add(Dense(500))
model.add(Activation('softmax'))
# 第一次建模用線性堆疊

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               392500    
_________________________________________________________________
activation (Activation)      (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
Total params: 893,500
Trainable params: 893,500
Non-trainable params: 0
__________________________________________________

In [11]:
model.layers

In [12]:
# 第二次建模用 list (每層分開) 
first_layer = [Dense(500, input_dim = 784), Activation('sigmoid')]
sencond_layer = [Dense(500), Activation('sigmoid')]
output_layer = [Dense(500), Activation('softmax')]

In [13]:
first_layer + sencond_layer + output_layer  # use list to connect three layers

In [14]:
model = Sequential(first_layer + sencond_layer + output_layer)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_3 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_4 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_5 (Activation)    (None, 500)               0         
Total params: 893,500
Trainable params: 893,500
Non-trainable params: 0
________________________________________________

In [16]:
# 第三次建模(前兩層包起來)
all_except_last = [Dense(500, input_dim = 784),
                   Activation('sigmoid'), 
                   Dense(500),
                   Activation('sigmoid')]

output_layer = [Dense(10), Activation('softmax')]

model_0_to_9 = Sequential(all_except_last + output_layer)
model_0_to_9.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_6 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                5010      
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
________________________________________________

In [17]:
model_0_to_9.load_weights('手寫辨識.h5')

In [18]:
new_output_layer = [Dense(2), Activation('softmax')]

model_0_to_1 = Sequential(all_except_last + new_output_layer)
model_0_to_1.summary()   # 可以發現前四行名字與上個 summary 是一樣的

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_6 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 1002      
_________________________________________________________________
activation_9 (Activation)    (None, 2)                 0         
Total params: 644,002
Trainable params: 644,002
Non-trainable params: 0
________________________________________________

In [19]:
for layer in all_except_last:
    layer.trainable = False  # 我們把它冷凍起來表示訓練時不用再訓練一次
    
model_0_to_1.summary() # 只有最下面 trainable params: 1002 沒有包含前四行

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_6 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 1002      
_________________________________________________________________
activation_9 (Activation)    (None, 2)                 0         
Total params: 644,002
Trainable params: 1,002
Non-trainable params: 643,000
____________________________________________

In [20]:
model_0_to_1.compile(loss = 'mse', optimizer = SGD(learning_rate = 0.1), metrics = ['accuracy'])

In [21]:
x_train_01.shape

(12665, 784)

In [22]:
y_train_01.shape

(12665, 2)

In [23]:
model_0_to_1.fit(x_train_01, y_train_01, batch_size = 100, epochs = 5)

Epoch 1/5
127/127 [==============================] - 6s 14ms/step - loss: 0.0144 - accuracy: 0.9799
Epoch 2/5
127/127 [==============================] - 2s 16ms/step - loss: 0.0035 - accuracy: 0.9968
Epoch 3/5
127/127 [==============================] - 3s 21ms/step - loss: 0.0029 - accuracy: 0.9972
Epoch 4/5
127/127 [==============================] - 3s 21ms/step - loss: 0.0026 - accuracy: 0.9975
Epoch 5/5
127/127 [==============================] - 2s 18ms/step - loss: 0.0025 - accuracy: 0.9973


In [24]:
score = model_0_to_1.evaluate(x_test_01, y_test_01)

67/67 [==============================] - 2s 17ms/step - loss: 0.0012 - accuracy: 0.9991


In [25]:
print('測試資料的loss:', score[0])
print('測試資料正確率:', score[1])

測試資料的loss: 0.0012303378898650408
測試資料正確率: 0.9990543723106384
